### TEXT PREPROCESSING

In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from ast import literal_eval
import re, string


txt = open('dataset.txt')
raw = txt.read()

def preprocess_word(w):
    # Removes punctuation
    translator = str.maketrans('', '', string.punctuation)
    punctuation = w.translate(translator)

    return punctuation


def preprocessing(x):
    # Returns a nested list of the processed sentences
    
    mentions = re.sub(r'@\w+',"", x) #removes mentions
    numbers = re.sub('[0-9]+', "", mentions) #removes numbers
    links = re.sub(r'http\S+', "", numbers) #removes links
    
    sentence_tokenized = sent_tokenize(links) #sentence tokenization - INACCURATE!!!
    
    lower = [[sent.lower()] for sent in sentence_tokenized] #lower text
     
    in_list = [word for sent in lower for word in sent]

    word_tokenized = [word_tokenize(sent) for sent in in_list]
    word_tokenized = [sent for sent in word_tokenized if sent] #word tokenization
    
    for _id, sent in enumerate(word_tokenized):
        word_tokenized[_id] =  [preprocess_word(w) for w in sent]
    
    words = [[word for word in sent if word != '' and word != 'rt'] for sent in word_tokenized] #removes useless words
    sentences = [sent for sent in words if sent] #removes empty sentences
    
    
    return sentences


text = preprocessing(raw)

df = pd.DataFrame(text)
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='welcome', index=False)
writer.save()

### BAG OF WORDS 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval

try:
    assert(literal_eval(str(text)) == text.copy())
except AssertionError:
    print('failed to convert')
    
final_str = [" ".join(x) for x in text]

train, test = train_test_split(final_str, test_size=0.2)

count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train)
print(train_counts)

### EMBEDDINGS 

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=text, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

vector = model.wv['bitch'] #returns numpy vector of a word

sims = model.wv.most_similar('bitch', topn=10) #returns similar words

print(vector)
print(sims)

In [ ]:
from sklearn.decomposition import IncrementalPCA  
from sklearn.manifold import TSNE                 
import numpy as np                                  


def reduce_dimensions(model):
    num_dimensions = 2 


    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key) 

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(autosize=True)
    plt.scatter(x_vals, y_vals)

    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

### SYNTAX

In [ ]:
def flatten_list(x):
#Takes a nested list and converts it into a list of elements
#where every sublist is a new element

    new_list = [] 
    
    for sent in text:
        sentences = " ".join(sent)
        new_list.append(sentences)
    
    return new_list

new = flatten_list(text)

In [ ]:
def dependency_parsing(x):
# Returns a nested list of syntactic labels

    nlp = spacy.load("en_core_web_sm")
    dependencies = []
    for sent in x:
        doc = nlp(sent)
        new_list = [token.dep_ for token in doc]
        dependencies.append(new_list)
        
    return dependencies

dep = dependency_parsing(new)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import spacy
import string
import pprint
%matplotlib inline

wordset = set()
for sentence in text:
    for word in sentence:
        wordset.add(word)

nlp = spacy.load("en_core_web_sm")

#Add every word as a node
base_graph = nx.Graph()
base_graph.add_nodes_from(wordset)
nx.draw(base_graph, with_labels=True)

graphs = {}
for sentence_id, sentence_contents in enumerate(new):
    sentence_graph = base_graph.copy()
    processed_sentence =  nlp(' '.join(new))
print('\n',processed_sentence)

In [ ]:
#Add edges between the nodes according to syntactic relations
for token in processed_sentence:
    nodeA = token.text
    nodeB = token.head.text
    print('\tadding edge between', nodeA, 'and', nodeB)
    sentence_graph.add_edge(nodeA, nodeB)
    sentence_representation =  nx.adjacency_matrix(sentence_graph) #sparse matrix
    print('\t sparse matrix has ',sentence_representation.count_nonzero(),'nonzero elements')
    graphs[sentence_id] = sentence_representation

In [ ]:
nx.draw(sentence_graph, with_labels=True)

In [ ]:
print(sentence_representation)

In [ ]:
print('\n\nsaving representations...')
import pickle
with open('sentence_representations_adjmat.p','wb') as fw:
    pickle.dump(graphs, fw)

### CLASSIFICATION

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

train, test = train_test_split(train_counts, test_size=0.5, random_state=0)
gnb = GaussianNB()
pred = gnb.fit(train, test)